## Analysing a sample of Body Mass Index

In this introductory notebook you will see how to:
        <ol>
        <li>Reading a File to a Dataframe</li>
        <li>Looking at the distribution of the data</li>
        <li>Creating a column for the Body Mass Index (BMI) using the formula</li>
        <li>Categorize the data</li>
        <li>Wanna Check your BMI and whether you are healthy?</li>
        </ol>
        

### 1. Reading a File to a Dataframe

To start, we read a file in csv format to a pandas dataframe, as in the real world this will often be where things kick off.

**Pandas** is a library which provides us with an abstraction (or data structure) called a ***Dataframe***, essentially this is a table. The good thing is that we get all the ***methods*** of this library for free!

You may think of a *method* as a function that will work for all instances of a specific data type.

For example, the pandas library has the **read_csv()** method which reads a csv file and converts it to a Dataframe. Another example, the Dataframe class has a built-in method called **head()** which shows you the header and first rows of your Dataframe.

This is a simplified explanation of what is happening behind the scenes in the code below. Try running it! 

In [ ]:
import pandas as pd

bmi_df = pd.read_csv(filepath_or_buffer="bmi.csv")
bmi_df.head()

You will note that we used an ***argument*** called *filepath_or_buffer* for the **read_csv()** method. This is where we specify the path to the file. If you place the file in the same directory as this notebook, then this code will work. Otherwise, you may use either the full or relative paths (but let's not get ahead of ourselves).

**Important**: when in doubt (will happen a lot) use google to browse a library's documentation. This will tell you what *methods*, *arguments* are available, as well as what is returned by each.

Try going to <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html">here</a> to see what *pandas.read_csv* can do. Examples (generally at the bottom of the page) in the documentation are often useful.

**Do not** read documentation like a book. In this example only the *filepath_or_buffer* is a mandatory argument, the other arguments are optional and may be ignored. You'll only really use them if your use case requires it, by then you'll know where to look.

### 2. Looking at the distribution of the data

We can get a quick look at statistics in a dataframe using the *describe()* method.

In [ ]:
bmi_df.describe()

This is a start but maybe we would like to look at how age and height are distributed for each gender.

In [ ]:
bmi_df.groupby("Gender").describe()

If you're paying attention you will note that in the code above we used the same *describe()*, but now we grouped the dataframe by gender using the *groupby()* method. (Examples [here](https://pandas.pydata.org/docs/getting_started/intro_tutorials/06_calculate_statistics.html))

This is better! But too much clutter, let's drop the Index column for this analysis.

In [ ]:
bmi_df = bmi_df[["Gender","Height","Weight"]] # dropping Index column, redeclaring the clean df
bmi_df.groupby("Gender").describe()

That's good. But let's get a graphical sense for these distributions.

In [ ]:
bmi_df.hist(column="Height", by="Gender", legend=True,bins=30)


In [ ]:
bmi_df.hist(column="Weight", by="Gender", legend=True,bins=30)

### 3. Creating a column for the Body Mass Index (BMI) using the formula

Formula: weight (kg) / [height (m)]2

Based on this formula, let us create a function that takes height and weight as arguments and returns the body mass index.

In [ ]:
def bmi_calculator(height:int, weight:int):
    return round( weight / ((height * 0.01)**2), 1)

Now let us create a column for each person in our dataframe using this function.

In [ ]:
bmi_df["BMI"] = bmi_calculator(height = bmi_df["Height"], weight = bmi_df["Weight"])

bmi_df.head()

Lastly, let's look at the data distribution grouped by gender using histograms.

In [ ]:
bmi_df.hist(column="BMI", by="Gender", bins=20)

### Categorize the data

Health studies have long studied this issue, so, to our "luck", there is already a table containing the different categories you can have with regard to BMI. The table below was taken from [here](https://www.verywellhealth.com/body-mass-index-bmi-5210240)

|IMC (kg/m²) |	Classificação |
|--|--|
|< 18,5 |	Magreza |
|18,5 a 24,9	| Peso normal |
|25 a 29,9	| Sobrepeso |
|30 a 34,9	| Obesidade grau I |
|35 a 40 |	Obesidade grau II |
|> 40 |	Obesidade grau III |

Let's write a simple function to convert the BMI values to their respective classification. A few if statements work here.

In [ ]:
def categorize(bmi:float):
    if bmi < 18.5:
        return "Magreza"
    elif bmi >= 18.5 and bmi < 25.0:
        return "Normal"
    elif  bmi >= 25.0 and bmi < 30.0:
        return "Sobrepeso"
    elif bmi >= 30.0 and bmi < 35.0:
        return "Obesidade grau I"
    elif bmi >= 35.0 and bmi < 40.0:
        return "Obesidade grau II"
    elif bmi >= 40.0:
        return "Obesidade grau III"

Now let's apply the function to each BMI value. We need to iterate (i.e., go through) each row, take the BMI and apply our function. Note that we are aware all values are of type float. Due to some Pandas nuisances, we first append the results to a list, then convert them to a pandas Series object and drop Nones, to then concatenate the results back into our original dataframe.

We call this "martelar código", but hey if it works it works. For simple exercises like this it is ok, for more serious things you would like to have a cleaner solution. I tend to do everything without pandas, using my own functions. But I am sure there are better ways to do it using the library. It is a trade off that one needs to evaluate according to the use case and application.

In [ ]:
series_list = []
for _, row in bmi_df.iterrows():
    series_list += [categorize(row["BMI"])] #appending to the list

series = pd.Series(series_list, name="classif").dropna() #dropping nones and converting list to pandas series object
bmi_df["classif"] = series # appending the series (same row length) to the dataframe as a column

bmi_df.head()

Let's plot the results in two histograms, with the distribution grouped by gender. Homework: Sort the column labels so they make more sense. You could possibly use an Enum object for this or find your own solution ;)

In [ ]:
bmi_df.hist(column="classif", by="Gender", sharex=True) # homework: sort the columns labels

### Wanna Check your BMI and whether you are healthy?

In [ ]:
height = int(input(f"what is your height ?"))
print(f"Your height is: {height}")


In [ ]:
weight = float(input(f"what is your weight ?"))
print(f"Your weight is: {weight}")

In [ ]:
my_bmi = bmi_calculator(height=height,weight=weight)
print(my_bmi)
print(f"Your BMI is: {categorize(my_bmi)}")